In [1]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw, rdFMCS
from openbabel import pybel
import pandas as pd
import re
from IPython.display import display
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import csv

In [2]:
# def convert_sdf_to_mol2(sdf_file, mol2_file):
#     sdf_mols = pybel.readfile("sdf", sdf_file)
#     out_mol2 = pybel.Outputfile("mol2", mol2_file, overwrite=False)

#     for mol in sdf_mols:
#         out_mol2.write(mol)

#     out_mol2.close()

# # Example usage
# convert_sdf_to_mol2('output.sdf', 'reconvert.mol2')

In [3]:
%run refactor_Torsion_Strain.py -i reconvert.mol2 -o reconvert_mol2.csv
#my conversion method doesn't work 

3 molecules finished reading. Calculating strain energy...
0 successful / 3 NA. Please check: reconvert_mol2.csv


[13:04:04] 

****
Pre-condition Violation
RingInfo not initialized
Violation occurred on line 129 in file /Users/runner/miniforge3/conda-bld/rdkit_1678104767387/work/Code/GraphMol/RingInfo.cpp
Failed Expression: df_init
****

[13:04:04] 

****
Pre-condition Violation
RingInfo not initialized
Violation occurred on line 129 in file /Users/runner/miniforge3/conda-bld/rdkit_1678104767387/work/Code/GraphMol/RingInfo.cpp
Failed Expression: df_init
****

[13:04:04] 

****
Pre-condition Violation
RingInfo not initialized
Violation occurred on line 129 in file /Users/runner/miniforge3/conda-bld/rdkit_1678104767387/work/Code/GraphMol/RingInfo.cpp
Failed Expression: df_init
****



In [7]:
%run refactor_Torsion_Strain.py -i test.mol2 -o test_mol2.csv
#converted in MOE by Francesco 

3 molecules finished reading. Calculating strain energy...
3 successful / 0 NA. Please check: test_mol2.csv


In [64]:
def create_dataframes(file1, file2):
    # Define a regular expression for the separator
    sep = r",(?![^[]*\])"  # Pass the compiled regular expression pattern as a string

    # Determine the origin from the filename
    origin1 = file1
    origin2 = file2

    dataframes = []

    with open(file1, "r") as f1, open(file2, "r") as f2:
        reader1 = csv.reader(f1)
        reader2 = csv.reader(f2)

        for line1, line2 in zip(reader1, reader2):
            # Define general column names
            column_names = [
                "Molecule_ID",
                "Total_Energy",
                "Energy_Lower_Bound",
                "Energy_Upper_Bound",
                "Num_Torsion_Patterns",
            ]

            # Define a dictionary mapping column names to data types
            dtypes = {
                "Molecule_ID": str,
                "Total_Energy": float,
                "Energy_Lower_Bound": float,
                "Energy_Upper_Bound": float,
                "Num_Torsion_Patterns": int,
                # Add more mappings as needed
            }

            # Calculate the number of torsion patterns
            num_torsion_patterns = (len(line1) - len(column_names)) // 10

            # Generate column names for each torsion pattern
            for i in range(1, num_torsion_patterns + 1):
                torsion_columns = [
                    f"Torsion_{i}_Energy",
                    f"Torsion_{i}_Energy_Lower_Bound",
                    f"Torsion_{i}_Energy_Upper_Bound",
                    f"Torsion_{i}_Atom_Indices",
                    f"Torsion_{i}_Dihedral_Angle",
                    f"Torsion_{i}_SMARTS_Pattern",
                    f"Torsion_{i}_Hierarchy_Class",
                    f"Torsion_{i}_Method",
                    f"Torsion_{i}_Indices_Reversed",
                    f"Torsion_{i}_Rule_Index",
                ]
                column_names.extend(torsion_columns)

                # Add data type mappings for the torsion columns
                torsion_dtypes = {
                    torsion_columns[0]: float,  # Energy
                    torsion_columns[1]: float,  # Energy Lower Bound
                    torsion_columns[2]: float,  # Energy Upper Bound
                    torsion_columns[3]: str,    # Atom Indices
                    torsion_columns[4]: float,  # Dihedral Angle
                    torsion_columns[5]: str,    # SMARTS Pattern
                    torsion_columns[6]: str,    # Hierarchy Class
                    torsion_columns[7]: str,    # Method
                    torsion_columns[8]: bool,   # Indices Reversed
                    torsion_columns[9]: int,    # Rule Index
                }
                dtypes.update(torsion_dtypes)

            # Slice the lines based on the number of torsion patterns
            line1 = line1[: len(column_names)]
            line2 = line2[: len(column_names)]

            # Convert lines to DataFrames
            df1 = pd.DataFrame([line1], columns=column_names)
            df2 = pd.DataFrame([line2], columns=column_names)

            # Force data types on columns
            df1 = df1.astype(dtypes)
            df2 = df2.astype(dtypes)

            # Add the origin to the DataFrames
            df1.insert(0, "Origin", origin1)
            df2.insert(0, "Origin", origin2)

            # Concatenate the new DataFrame with df and drop the index
            df = pd.concat([df1, df2], ignore_index=True)

            dataframes.append(df)

            # Reset column names for next line
            column_names = column_names[:5]

    return dataframes

In [65]:
# Prevent truncating of dataframe display
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df_1, df_2, df_3 = create_dataframes("test_mol2.csv", "mol2.csv")
display(df_1)
display(df_2)
display(df_3)

,Origin,Molecule_ID,Total_Energy,Energy_Lower_Bound,Energy_Upper_Bound,Num_Torsion_Patterns,Torsion_1_Energy,Torsion_1_Energy_Lower_Bound,Torsion_1_Energy_Upper_Bound,Torsion_1_Atom_Indices,Torsion_1_Dihedral_Angle,Torsion_1_SMARTS_Pattern,Torsion_1_Hierarchy_Class,Torsion_1_Method,Torsion_1_Indices_Reversed,Torsion_1_Rule_Index,Torsion_2_Energy,Torsion_2_Energy_Lower_Bound,Torsion_2_Energy_Upper_Bound,Torsion_2_Atom_Indices,Torsion_2_Dihedral_Angle,Torsion_2_SMARTS_Pattern,Torsion_2_Hierarchy_Class,Torsion_2_Method,Torsion_2_Indices_Reversed,Torsion_2_Rule_Index,Torsion_3_Energy,Torsion_3_Energy_Lower_Bound,Torsion_3_Energy_Upper_Bound,Torsion_3_Atom_Indices,Torsion_3_Dihedral_Angle,Torsion_3_SMARTS_Pattern,Torsion_3_Hierarchy_Class,Torsion_3_Method,Torsion_3_Indices_Reversed,Torsion_3_Rule_Index,Torsion_4_Energy,Torsion_4_Energy_Lower_Bound,Torsion_4_Energy_Upper_Bound,Torsion_4_Atom_Indices,Torsion_4_Dihedral_Angle,Torsion_4_SMARTS_Pattern,Torsion_4_Hierarchy_Class,Torsion_4_Method,Torsion_4_Indices_Reversed,Torsion_4_Rule_Index,Torsion_5_Energy,Torsion_5_Energy_Lower_Bound,Torsion_5_Energy_Upper_Bound,Torsion_5_Atom_Indices,Torsion_5_Dihedral_Angle,Torsion_5_SMARTS_Pattern,Torsion_5_Hierarchy_Class,Torsion_5_Method,Torsion_5_Indices_Reversed,Torsion_5_Rule_Index,Torsion_6_Energy,Torsion_6_Energy_Lower_Bound,Torsion_6_Energy_Upper_Bound,Torsion_6_Atom_Indices,Torsion_6_Dihedral_Angle,Torsion_6_SMARTS_Pattern,Torsion_6_Hierarchy_Class,Torsion_6_Method,Torsion_6_Indices_Reversed,Torsion_6_Rule_Index,Torsion_7_Energy,Torsion_7_Energy_Lower_Bound,Torsion_7_Energy_Upper_Bound,Torsion_7_Atom_Indices,Torsion_7_Dihedral_Angle,Torsion_7_SMARTS_Pattern,Torsion_7_Hierarchy_Class,Torsion_7_Method,Torsion_7_Indices_Reversed,Torsion_7_Rule_Index,Torsion_8_Energy,Torsion_8_Energy_Lower_Bound,Torsion_8_Energy_Upper_Bound,Torsion_8_Atom_Indices,Torsion_8_Dihedral_Angle,Torsion_8_SMARTS_Pattern,Torsion_8_Hierarchy_Class,Torsion_8_Method,Torsion_8_Indices_Reversed,Torsion_8_Rule_Index
0,test_mol2.csv,ZINC000191583186,5.617653,5.069657,6.317096,4,1.513683,1.470869,1.559806,"[15, 13, 16, 17]",-79.984285,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,True,8,1.161721,1.012904,1.357245,"[16, 13, 10, 6]",-160.014813,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,True,7,1.160731,1.012061,1.356042,"[8, 6, 10, 13]",-160.030648,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,True,5,1.038466,0.888659,1.241182,"[19, 18, 17, 16]",0.219272,[a:1][c:2]!@[CX4H2:3][CX4:4],specific,exact,True,1,0.346130,0.339974,0.352348,"[5, 6, 8, 9]",59.977294,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,True,3,0.345047,0.338908,0.351246,"[4, 5, 6, 8]",60.003038,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,True,0,0.041449,0.028447,0.054747,"[22, 23, 24, 26]",0.407968,[a:1][a:2]!@[NX3:3][!#1:4],specific,exact,True,6,0.006961,-0.022062,0.037426,"[27, 4, 5, 6]",90.015458,[a:1][c:2]!@[CX4H2:3][CX4:4],specific,exact,True,2
1,mol2.csv,ZINC000191583186,5.621730,5.072461,NaN,4,1.514902,1.472042,1.561076,"[15, 13, 16, 17]",-79.998769,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,True,8,1.162950,1.013876,NaN,"[16, 13, 10, 6]",-159.996786,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,True,7,1.162767,1.013766,NaN,"[8, 6, 10, 13]",-159.998720,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,True,5,1.040802,0.890317,1.245260,"[19, 18, 17, 16]",0.273439,[a:1][c:2]!@[CX4H2:3][CX4:4],specific,exact,True,3,0.345286,0.339143,0.351490,"[4, 5, 6, 8]",59.993486,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,True,1,0.345222,0.339081,0.351424,"[5, 6, 8, 9]",60.008310,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,True,0,0.042865,0.029821,0.056206,"[22, 23, 24, 26]",0.445858,[a:1][a:2]!@[NX3:3][!#1:4],specific,exact,True,6,0.006708,-0.022299,0.037155,"[27, 4, 5, 6]",89.992702,[a:1][c:2]!@[CX4H2:3][CX4:4],specific,exact,True,2


,Origin,Molecule_ID,Total_Energy,Energy_Lower_Bound,Energy_Upper_Bound,Num_Torsion_Patterns,Torsion_1_Energy,Torsion_1_Energy_Lower_Bound,Torsion_1_Energy_Upper_Bound,Torsion_1_Atom_Indices,Torsion_1_Dihedral_Angle,Torsion_1_SMARTS_Pattern,Torsion_1_Hierarchy_Class,Torsion_1_Method,Torsion_1_Indices_Reversed,Torsion_1_Rule_Index,Torsion_2_Energy,Torsion_2_Energy_Lower_Bound,Torsion_2_Energy_Upper_Bound,Torsion_2_Atom_Indices,Torsion_2_Dihedral_Angle,Torsion_2_SMARTS_Pattern,Torsion_2_Hierarchy_Class,Torsion_2_Method,Torsion_2_Indices_Reversed,Torsion_2_Rule_Index,Torsion_3_Energy,Torsion_3_Energy_Lower_Bound,Torsion_3_Energy_Upper_Bound,Torsion_3_Atom_Indices,Torsion_3_Dihedral_Angle,Torsion_3_SMARTS_Pattern,Torsion_3_Hierarchy_Class,Torsion_3_Method,Torsion_3_Indices_Reversed,Torsion_3_Rule_Index,Torsion_4_Energy,Torsion_4_Energy_Lower_Bound,Torsion_4_Energy_Upper_Bound,Torsion_4_Atom_Indices,Torsion_4_Dihedral_Angle,Torsion_4_SMARTS_Pattern,Torsion_4_Hierarchy_Class,Torsion_4_Method,Torsion_4_Indices_Reversed,Torsion_4_Rule_Index,Torsion_5_Energy,Torsion_5_Energy_Lower_Bound,Torsion_5_Energy_Upper_Bound,Torsion_5_Atom_Indices,Torsion_5_Dihedral_Angle,Torsion_5_SMARTS_Pattern,Torsion_5_Hierarchy_Class,Torsion_5_Method,Torsion_5_Indices_Reversed,Torsion_5_Rule_Index,Torsion_6_Energy,Torsion_6_Energy_Lower_Bound,Torsion_6_Energy_Upper_Bound,Torsion_6_Atom_Indices,Torsion_6_Dihedral_Angle,Torsion_6_SMARTS_Pattern,Torsion_6_Hierarchy_Class,Torsion_6_Method,Torsion_6_Indices_Reversed,Torsion_6_Rule_Index
0,test_mol2.csv,ZINC000159533726,3.449552,2.437370,NaN,4,2.120178,1.283001,NaN,"[16, 1, 3, 4]",-120.010117,[NH1:1][CX4:2]!@[CX3:3]=[O:4],specific,exact,True,0,0.876237,0.774808,0.997498,"[14, 6, 5, 3]",-149.932006,"[a:1][a:2]!@[NH1:3][C,c:4]",specific,exact,True,3,0.396174,0.389351,0.403077,"[16, 19, 20, 22]",-61.406806,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,True,2,0.027623,0.015021,0.040495,"[12, 8, 9, 11]",0.037864,[a:1][a:2]!@[NX3:3][!#1:4],specific,exact,True,5,0.013965,-0.009981,0.038531,"[3, 1, 16, 19]",-179.942283,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,True,6,0.012063,-0.011770,0.03650,"[20, 19, 16, 1]",179.980227,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,True,1
1,mol2.csv,ZINC000159533726,3.441375,2.429431,NaN,4,2.120636,1.283213,NaN,"[16, 1, 3, 4]",-120.006155,[NH1:1][CX4:2]!@[CX3:3]=[O:4],specific,exact,True,0,0.875247,0.773987,0.996252,"[14, 6, 5, 3]",-150.004143,"[a:1][a:2]!@[NH1:3][C,c:4]",specific,exact,True,3,0.395620,0.388804,0.402515,"[16, 19, 20, 22]",-61.389633,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,True,2,0.026946,0.014363,0.039797,"[12, 8, 9, 11]",0.019719,[a:1][a:2]!@[NX3:3][!#1:4],specific,exact,True,5,0.011319,-0.012464,0.035700,"[0, 1, 16, 18]",-179.992537,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,True,6,0.011141,-0.012632,0.03551,"[20, 19, 16, 1]",179.996269,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,True,1


,Origin,Molecule_ID,Total_Energy,Energy_Lower_Bound,Energy_Upper_Bound,Num_Torsion_Patterns,Torsion_1_Energy,Torsion_1_Energy_Lower_Bound,Torsion_1_Energy_Upper_Bound,Torsion_1_Atom_Indices,Torsion_1_Dihedral_Angle,Torsion_1_SMARTS_Pattern,Torsion_1_Hierarchy_Class,Torsion_1_Method,Torsion_1_Indices_Reversed,Torsion_1_Rule_Index,Torsion_2_Energy,Torsion_2_Energy_Lower_Bound,Torsion_2_Energy_Upper_Bound,Torsion_2_Atom_Indices,Torsion_2_Dihedral_Angle,Torsion_2_SMARTS_Pattern,Torsion_2_Hierarchy_Class,Torsion_2_Method,Torsion_2_Indices_Reversed,Torsion_2_Rule_Index,Torsion_3_Energy,Torsion_3_Energy_Lower_Bound,Torsion_3_Energy_Upper_Bound,Torsion_3_Atom_Indices,Torsion_3_Dihedral_Angle,Torsion_3_SMARTS_Pattern,Torsion_3_Hierarchy_Class,Torsion_3_Method,Torsion_3_Indices_Reversed,Torsion_3_Rule_Index,Torsion_4_Energy,Torsion_4_Energy_Lower_Bound,Torsion_4_Energy_Upper_Bound,Torsion_4_Atom_Indices,Torsion_4_Dihedral_Angle,Torsion_4_SMARTS_Pattern,Torsion_4_Hierarchy_Class,Torsion_4_Method,Torsion_4_Indices_Reversed,Torsion_4_Rule_Index,Torsion_5_Energy,Torsion_5_Energy_Lower_Bound,Torsion_5_Energy_Upper_Bound,Torsion_5_Atom_Indices,Torsion_5_Dihedral_Angle,Torsion_5_SMARTS_Pattern,Torsion_5_Hierarchy_Class,Torsion_5_Method,Torsion_5_Indices_Reversed,Torsion_5_Rule_Index
0,test_mol2.csv,ZINC000151228439,3.189726,2.774645,3.717202,4,1.225324,1.061785,1.447688,"[16, 1, 3, 6]",160.013072,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,True,2,0.865692,0.798266,0.941471,"[3, 1, 16, 26]",-120.363633,[!#1:1][CX4:2]!@[CX3H0:3][!#1:4],specific,exact,True,3,0.706278,0.564914,0.891918,"[11, 10, 7, 9]",-120.046371,"[a:1][c:2]!@[CX4H0:3][N,O:4]",specific,exact,True,1,0.345670,0.339522,0.351878,"[3, 6, 7, 8]",60.021766,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,True,0,0.035671,0.022836,0.048791,"[20, 19, 18, 25]",0.253298,[a:1][a:2]!@[NX3:3][!#1:4],specific,exact,True,5
1,mol2.csv,ZINC000151228439,3.192153,2.776743,3.720130,4,1.226140,1.062470,1.448703,"[16, 1, 3, 6]",160.000329,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,True,2,0.865586,0.798172,0.941350,"[3, 1, 16, 26]",-120.315336,[!#1:1][CX4:2]!@[CX3H0:3][!#1:4],specific,exact,True,3,0.707273,0.565726,0.893202,"[11, 10, 7, 9]",-120.002110,"[a:1][c:2]!@[CX4H0:3][N,O:4]",specific,exact,True,1,0.345013,0.338874,0.351211,"[3, 6, 7, 8]",60.001999,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,True,0,0.037163,0.024285,0.050329,"[20, 19, 18, 25]",0.293234,[a:1][a:2]!@[NX3:3][!#1:4],specific,exact,True,5


In [66]:
def select_unique_columns_rounded(df):
    # Reset the index of df
    df = df.reset_index(drop=True)

    # Round the DataFrame to the 8th decimal place
    df_rounded = df.round(8)

    # Select the first two columns
    # of df_rounded
    first_two_columns = df_rounded.iloc[:, :2]

    # Select the rest of the columns where the rows have different values
    unique_columns = df_rounded.iloc[:, 2:].loc[:, df_rounded.iloc[:, 2:].nunique() > 1]

    # Concatenate the first two columns and the unique columns
    df_unique = pd.concat([first_two_columns, unique_columns], axis=1)

    # Return the new DataFrame
    return df_unique

In [67]:
df_1_r8_unique = select_unique_columns_rounded(df_1)
df_2_r8_unique = select_unique_columns_rounded(df_2)
df_3_r8_unique = select_unique_columns_rounded(df_3)
print("Here are the shapes of the differences in the dataframes:")
print(df_1_r8_unique.shape, df_2_r8_unique.shape, df_3_r8_unique.shape)

Here are the shapes of the differences in the dataframes:
(2, 38) (2, 28) (2, 25)


In [68]:
display(df_1_r8_unique)
display(df_2_r8_unique)
display(df_3_r8_unique)

,Origin,Molecule_ID,Total_Energy,Energy_Lower_Bound,Torsion_1_Energy,Torsion_1_Energy_Lower_Bound,Torsion_1_Energy_Upper_Bound,Torsion_1_Dihedral_Angle,Torsion_2_Energy,Torsion_2_Energy_Lower_Bound,Torsion_2_Dihedral_Angle,Torsion_3_Energy,Torsion_3_Energy_Lower_Bound,Torsion_3_Dihedral_Angle,Torsion_4_Energy,Torsion_4_Energy_Lower_Bound,Torsion_4_Energy_Upper_Bound,Torsion_4_Dihedral_Angle,Torsion_4_Rule_Index,Torsion_5_Energy,Torsion_5_Energy_Lower_Bound,Torsion_5_Energy_Upper_Bound,Torsion_5_Atom_Indices,Torsion_5_Dihedral_Angle,Torsion_5_Rule_Index,Torsion_6_Energy,Torsion_6_Energy_Lower_Bound,Torsion_6_Energy_Upper_Bound,Torsion_6_Atom_Indices,Torsion_6_Dihedral_Angle,Torsion_7_Energy,Torsion_7_Energy_Lower_Bound,Torsion_7_Energy_Upper_Bound,Torsion_7_Dihedral_Angle,Torsion_8_Energy,Torsion_8_Energy_Lower_Bound,Torsion_8_Energy_Upper_Bound,Torsion_8_Dihedral_Angle
0,test_mol2.csv,ZINC000191583186,5.617653,5.069657,1.513683,1.470869,1.559806,-79.984285,1.161721,1.012904,-160.014813,1.160731,1.012061,-160.030648,1.038466,0.888659,1.241182,0.219272,1,0.346130,0.339974,0.352348,"[5, 6, 8, 9]",59.977294,3,0.345047,0.338908,0.351246,"[4, 5, 6, 8]",60.003038,0.041449,0.028447,0.054747,0.407968,0.006961,-0.022062,0.037426,90.015458
1,mol2.csv,ZINC000191583186,5.621730,5.072461,1.514901,1.472042,1.561076,-79.998769,1.162950,1.013876,-159.996786,1.162767,1.013766,-159.998720,1.040802,0.890317,1.245260,0.273439,3,0.345286,0.339143,0.351490,"[4, 5, 6, 8]",59.993486,1,0.345222,0.339081,0.351424,"[5, 6, 8, 9]",60.008310,0.042865,0.029821,0.056206,0.445858,0.006708,-0.022299,0.037155,89.992702


,Origin,Molecule_ID,Total_Energy,Energy_Lower_Bound,Torsion_1_Energy,Torsion_1_Energy_Lower_Bound,Torsion_1_Dihedral_Angle,Torsion_2_Energy,Torsion_2_Energy_Lower_Bound,Torsion_2_Energy_Upper_Bound,Torsion_2_Dihedral_Angle,Torsion_3_Energy,Torsion_3_Energy_Lower_Bound,Torsion_3_Energy_Upper_Bound,Torsion_3_Dihedral_Angle,Torsion_4_Energy,Torsion_4_Energy_Lower_Bound,Torsion_4_Energy_Upper_Bound,Torsion_4_Dihedral_Angle,Torsion_5_Energy,Torsion_5_Energy_Lower_Bound,Torsion_5_Energy_Upper_Bound,Torsion_5_Atom_Indices,Torsion_5_Dihedral_Angle,Torsion_6_Energy,Torsion_6_Energy_Lower_Bound,Torsion_6_Energy_Upper_Bound,Torsion_6_Dihedral_Angle
0,test_mol2.csv,ZINC000159533726,3.449552,2.437370,2.120178,1.283001,-120.010117,0.876237,0.774808,0.997498,-149.932006,0.396174,0.389351,0.403077,-61.406806,0.027623,0.015021,0.040495,0.037864,0.013965,-0.009981,0.038531,"[3, 1, 16, 19]",-179.942283,0.012063,-0.011770,0.03650,179.980227
1,mol2.csv,ZINC000159533726,3.441375,2.429431,2.120636,1.283214,-120.006155,0.875247,0.773987,0.996252,-150.004143,0.395620,0.388804,0.402515,-61.389633,0.026946,0.014363,0.039797,0.019719,0.011319,-0.012464,0.035700,"[0, 1, 16, 18]",-179.992537,0.011141,-0.012632,0.03551,179.996269


,Origin,Molecule_ID,Total_Energy,Energy_Lower_Bound,Energy_Upper_Bound,Torsion_1_Energy,Torsion_1_Energy_Lower_Bound,Torsion_1_Energy_Upper_Bound,Torsion_1_Dihedral_Angle,Torsion_2_Energy,Torsion_2_Energy_Lower_Bound,Torsion_2_Energy_Upper_Bound,Torsion_2_Dihedral_Angle,Torsion_3_Energy,Torsion_3_Energy_Lower_Bound,Torsion_3_Energy_Upper_Bound,Torsion_3_Dihedral_Angle,Torsion_4_Energy,Torsion_4_Energy_Lower_Bound,Torsion_4_Energy_Upper_Bound,Torsion_4_Dihedral_Angle,Torsion_5_Energy,Torsion_5_Energy_Lower_Bound,Torsion_5_Energy_Upper_Bound,Torsion_5_Dihedral_Angle
0,test_mol2.csv,ZINC000151228439,3.189726,2.774645,3.717202,1.225324,1.061785,1.447688,160.013072,0.865692,0.798266,0.941471,-120.363633,0.706278,0.564914,0.891918,-120.046371,0.345670,0.339522,0.351878,60.021766,0.035671,0.022836,0.048791,0.253298
1,mol2.csv,ZINC000151228439,3.192153,2.776743,3.720130,1.226140,1.062470,1.448703,160.000329,0.865586,0.798172,0.941350,-120.315336,0.707273,0.565726,0.893202,-120.002110,0.345013,0.338874,0.351211,60.001999,0.037163,0.024285,0.050329,0.293234


In [69]:
def select_identical_object_columns(df):
    # Convert list objects to strings
    for col in df.columns:
        if df[col].apply(type).eq(list).any():
            df[col] = df[col].apply(str)

    # Select the first two columns
    first_two_columns = df.iloc[:, :2]

    # Select columns where dtype is object
    object_columns = df.select_dtypes(include=["object"])

    # Select columns where all values are the same
    identical_object_columns = object_columns.loc[:, (object_columns.nunique() == 1)]

    # Concatenate the first two columns and the identical object columns
    df_identical = pd.concat([first_two_columns, identical_object_columns], axis=1)

    # Return the new DataFrame
    return df_identical


In [70]:
df_1_identical = select_identical_object_columns(df_1)
display(df_1_identical)
df_2_identical = select_identical_object_columns(df_2)
display(df_2_identical)
df_3_identical = select_identical_object_columns(df_3)
display(df_3_identical)

,Origin,Molecule_ID,Molecule_ID,Torsion_1_Atom_Indices,Torsion_1_SMARTS_Pattern,Torsion_1_Hierarchy_Class,Torsion_1_Method,Torsion_2_Atom_Indices,Torsion_2_SMARTS_Pattern,Torsion_2_Hierarchy_Class,Torsion_2_Method,Torsion_3_Atom_Indices,Torsion_3_SMARTS_Pattern,Torsion_3_Hierarchy_Class,Torsion_3_Method,Torsion_4_Atom_Indices,Torsion_4_SMARTS_Pattern,Torsion_4_Hierarchy_Class,Torsion_4_Method,Torsion_5_SMARTS_Pattern,Torsion_5_Hierarchy_Class,Torsion_5_Method,Torsion_6_SMARTS_Pattern,Torsion_6_Hierarchy_Class,Torsion_6_Method,Torsion_7_Atom_Indices,Torsion_7_SMARTS_Pattern,Torsion_7_Hierarchy_Class,Torsion_7_Method,Torsion_8_Atom_Indices,Torsion_8_SMARTS_Pattern,Torsion_8_Hierarchy_Class,Torsion_8_Method
0,test_mol2.csv,ZINC000191583186,ZINC000191583186,"[15, 13, 16, 17]",[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,"[16, 13, 10, 6]",[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,"[8, 6, 10, 13]",[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,"[19, 18, 17, 16]",[a:1][c:2]!@[CX4H2:3][CX4:4],specific,exact,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,"[22, 23, 24, 26]",[a:1][a:2]!@[NX3:3][!#1:4],specific,exact,"[27, 4, 5, 6]",[a:1][c:2]!@[CX4H2:3][CX4:4],specific,exact
1,mol2.csv,ZINC000191583186,ZINC000191583186,"[15, 13, 16, 17]",[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,"[16, 13, 10, 6]",[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,"[8, 6, 10, 13]",[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,"[19, 18, 17, 16]",[a:1][c:2]!@[CX4H2:3][CX4:4],specific,exact,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,"[22, 23, 24, 26]",[a:1][a:2]!@[NX3:3][!#1:4],specific,exact,"[27, 4, 5, 6]",[a:1][c:2]!@[CX4H2:3][CX4:4],specific,exact


,Origin,Molecule_ID,Molecule_ID,Torsion_1_Atom_Indices,Torsion_1_SMARTS_Pattern,Torsion_1_Hierarchy_Class,Torsion_1_Method,Torsion_2_Atom_Indices,Torsion_2_SMARTS_Pattern,Torsion_2_Hierarchy_Class,Torsion_2_Method,Torsion_3_Atom_Indices,Torsion_3_SMARTS_Pattern,Torsion_3_Hierarchy_Class,Torsion_3_Method,Torsion_4_Atom_Indices,Torsion_4_SMARTS_Pattern,Torsion_4_Hierarchy_Class,Torsion_4_Method,Torsion_5_SMARTS_Pattern,Torsion_5_Hierarchy_Class,Torsion_5_Method,Torsion_6_Atom_Indices,Torsion_6_SMARTS_Pattern,Torsion_6_Hierarchy_Class,Torsion_6_Method
0,test_mol2.csv,ZINC000159533726,ZINC000159533726,"[16, 1, 3, 4]",[NH1:1][CX4:2]!@[CX3:3]=[O:4],specific,exact,"[14, 6, 5, 3]","[a:1][a:2]!@[NH1:3][C,c:4]",specific,exact,"[16, 19, 20, 22]",[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,"[12, 8, 9, 11]",[a:1][a:2]!@[NX3:3][!#1:4],specific,exact,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,"[20, 19, 16, 1]",[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact
1,mol2.csv,ZINC000159533726,ZINC000159533726,"[16, 1, 3, 4]",[NH1:1][CX4:2]!@[CX3:3]=[O:4],specific,exact,"[14, 6, 5, 3]","[a:1][a:2]!@[NH1:3][C,c:4]",specific,exact,"[16, 19, 20, 22]",[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,"[12, 8, 9, 11]",[a:1][a:2]!@[NX3:3][!#1:4],specific,exact,[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,"[20, 19, 16, 1]",[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact


,Origin,Molecule_ID,Molecule_ID,Torsion_1_Atom_Indices,Torsion_1_SMARTS_Pattern,Torsion_1_Hierarchy_Class,Torsion_1_Method,Torsion_2_Atom_Indices,Torsion_2_SMARTS_Pattern,Torsion_2_Hierarchy_Class,Torsion_2_Method,Torsion_3_Atom_Indices,Torsion_3_SMARTS_Pattern,Torsion_3_Hierarchy_Class,Torsion_3_Method,Torsion_4_Atom_Indices,Torsion_4_SMARTS_Pattern,Torsion_4_Hierarchy_Class,Torsion_4_Method,Torsion_5_Atom_Indices,Torsion_5_SMARTS_Pattern,Torsion_5_Hierarchy_Class,Torsion_5_Method
0,test_mol2.csv,ZINC000151228439,ZINC000151228439,"[16, 1, 3, 6]",[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,"[3, 1, 16, 26]",[!#1:1][CX4:2]!@[CX3H0:3][!#1:4],specific,exact,"[11, 10, 7, 9]","[a:1][c:2]!@[CX4H0:3][N,O:4]",specific,exact,"[3, 6, 7, 8]",[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,"[20, 19, 18, 25]",[a:1][a:2]!@[NX3:3][!#1:4],specific,exact
1,mol2.csv,ZINC000151228439,ZINC000151228439,"[16, 1, 3, 6]",[*:1]~[CX4:2]!@[NX4:3]~[*:4],general,exact,"[3, 1, 16, 26]",[!#1:1][CX4:2]!@[CX3H0:3][!#1:4],specific,exact,"[11, 10, 7, 9]","[a:1][c:2]!@[CX4H0:3][N,O:4]",specific,exact,"[3, 6, 7, 8]",[!#1:1][CX4:2]!@[CX4:3][!#1:4],specific,exact,"[20, 19, 18, 25]",[a:1][a:2]!@[NX3:3][!#1:4],specific,exact


In [71]:
def select_unique_object_columns(df):
    # Convert list objects to strings
    for col in df.columns:
        if df[col].apply(type).eq(list).any():
            df[col] = df[col].apply(str)

    # Select the first two columns
    first_two_columns = df.iloc[:, :2]

    # Select columns where dtype is object
    object_columns = df.select_dtypes(include=["object"])

    # Select columns where all values are unique
    unique_object_columns = object_columns.loc[:, (object_columns.nunique() == df.shape[0])]

    # Concatenate the first two columns and the unique object columns
    df_unique = pd.concat([first_two_columns, unique_object_columns], axis=1)

    # Return the new DataFrame
    return df_unique


In [72]:
df_1_unique = select_unique_object_columns(df_1)
display(df_1_unique)
df_2_unique = select_unique_object_columns(df_2)
display(df_2_unique)
df_3_unique = select_unique_object_columns(df_3)
display(df_3_unique)

,Origin,Molecule_ID,Origin,Torsion_5_Atom_Indices,Torsion_6_Atom_Indices
0,test_mol2.csv,ZINC000191583186,test_mol2.csv,"[5, 6, 8, 9]","[4, 5, 6, 8]"
1,mol2.csv,ZINC000191583186,mol2.csv,"[4, 5, 6, 8]","[5, 6, 8, 9]"


,Origin,Molecule_ID,Origin,Torsion_5_Atom_Indices
0,test_mol2.csv,ZINC000159533726,test_mol2.csv,"[3, 1, 16, 19]"
1,mol2.csv,ZINC000159533726,mol2.csv,"[0, 1, 16, 18]"


,Origin,Molecule_ID,Origin
0,test_mol2.csv,ZINC000151228439,test_mol2.csv
1,mol2.csv,ZINC000151228439,mol2.csv


Previous debugging scripts need to be refactored for I/O stuff to be easier to use but I can hardcode the paths to do this faster:

In [73]:
import refactor_TL_Functions_debug as rtf

# Load the molecules with Francesco's MOE mol2 file
names_custom, mols_custom = rtf.Mol2MolSupplier("test.mol2")

# Load the molecules with Mol2MolSupplier
names_mol2, mols_mol2 = rtf.Mol2MolSupplier("example.mol2")


def compare_lists(list1, list2, name, property_name):
    # Check if the lengths of the outer lists are the same
    if len(list1) != len(list2):
        print(f"Different number of elements in {property_name} for molecule {name}")
    else:
        # Iterate over the lists and compare each element
        difference_found = False
        for i in range(len(list1)):
            if list1[i] != list2[i]:
                print(
                    f"Difference found in element {i} of {property_name} for molecule {name}"
                )
                print(f"TL_MOE_mol2: {list1[i]}, TL_mol2: {list2[i]}")
                difference_found = True
        if not difference_found:
            print(f"No differences found in {property_name} for molecule {name}")


for name in names_custom:
    mol_custom = mols_custom[name]
    mol_mol2 = mols_mol2[name]

    print(f"Molecule in MOE mol2: {name}")
    TL_custom = rtf.TL_lookup(mol_custom)
    print(f"Molecule in mol2: {name}")
    TL_mol2 = rtf.TL_lookup(mol_mol2)

    compare_lists(TL_custom.indeces, TL_mol2.indeces, name, "indeces")
    # compare_lists(TL_custom.angles, TL_mol2.angles, name, "angles")
    compare_lists(TL_custom.smarts, TL_mol2.smarts, name, "smarts")
    # compare_lists(TL_custom.hc, TL_mol2.hc, name, "hc")
    # compare_lists(TL_custom.methods, TL_mol2.methods, name, "methods")
    # compare_lists(TL_custom.E, TL_mol2.E, name, "E")
    # compare_lists(TL_custom.CI_l, TL_mol2.CI_l, name, "CI_l")
    # compare_lists(TL_custom.CI_u, TL_mol2.CI_u, name, "CI_u")
    # compare_lists(TL_custom.flags, TL_mol2.flags, name, "flags")

Molecule in MOE mol2: ZINC000191583186
Molecule in mol2: ZINC000191583186
No differences found in indeces for molecule ZINC000191583186
No differences found in smarts for molecule ZINC000191583186
Molecule in MOE mol2: ZINC000159533726
Molecule in mol2: ZINC000159533726
Difference found in element 5 of indeces for molecule ZINC000159533726
TL_MOE_mol2: [3, 1, 16, 19], TL_mol2: [0, 1, 16, 18]
No differences found in smarts for molecule ZINC000159533726
Molecule in MOE mol2: ZINC000151228439
Molecule in mol2: ZINC000151228439
No differences found in indeces for molecule ZINC000151228439
No differences found in smarts for molecule ZINC000151228439


In [74]:
import refactor_TL_Functions_debug as rtf

# Load the molecules with Francesco's MOE mol2 file
names_custom, mols_custom = rtf.Mol2MolSupplier("test.mol2")

# Load the molecules with Mol2MolSupplier
names_mol2, mols_mol2 = rtf.Mol2MolSupplier("example.mol2")

for name in names_custom:
    mol_custom = mols_custom[name]
    mol_mol2 = mols_mol2[name]

    print(f"Molecule in MOE mol2: {name}")
    TL_custom = rtf.TL_lookup(mol_custom)

    # Store the contents of match_debug_list in a variable
    match_debug_list_custom = list(rtf.match_debug_list)

    # Clear match_debug_list at the end of each loop iteration
    rtf.match_debug_list.clear()

    print(f"Molecule in mol2: {name}")
    TL_mol2 = rtf.TL_lookup(mol_mol2)

    # Compare the elements of match_debug_list_custom and match_debug_list
    for custom, mol2 in zip(match_debug_list_custom, rtf.match_debug_list):
        if custom != mol2:
            print(f"Difference found: {custom} != {mol2}")

    # Clear match_debug_list at the end of each loop iteration
    rtf.match_debug_list.clear()

Molecule in MOE mol2: ZINC000191583186
Molecule in mol2: ZINC000191583186
Difference found: (16, 17, 18, 19) != (16, 17, 18, 23)
Difference found: (16, 17, 18, 23) != (16, 17, 18, 19)
Molecule in MOE mol2: ZINC000159533726
Molecule in mol2: ZINC000159533726
Difference found: (3, 5, 6, 7) != (3, 5, 6, 14)
Difference found: (3, 5, 6, 14) != (3, 5, 6, 7)
Difference found: (3, 5, 6, 7) != (3, 5, 6, 14)
Difference found: (3, 5, 6, 14) != (3, 5, 6, 7)
Difference found: (16, 19, 20, 22) != (16, 19, 20, 25)
Difference found: (16, 19, 20, 25) != (16, 19, 20, 22)
Difference found: (16, 19, 20, 22) != (16, 19, 20, 25)
Difference found: (16, 19, 20, 25) != (16, 19, 20, 22)
Molecule in MOE mol2: ZINC000151228439
Molecule in mol2: ZINC000151228439
Difference found: (0, 1, 16, 17) != (0, 1, 16, 26)
Difference found: (0, 1, 16, 26) != (0, 1, 16, 17)
Difference found: (3, 1, 16, 17) != (3, 1, 16, 26)
Difference found: (3, 1, 16, 26) != (3, 1, 16, 17)
Difference found: (6, 7, 10, 11) != (6, 7, 10, 15)
D

There's still some ordering differences I think? Just gonna upload this for now so you can see it and keep looking at it